# Classificação Multilabel de textos: múltiplos contextos com NLP
***
Este projeto é baseado no curso <strong>Alura - Classificação Multilabel de textos: múltiplos contextos com NLP</strong>. O objetivo é criar uma classificação multilabel de textos e para isso precisamos relembrar a clasifficação binária, classificação multiclasses e classificação multilabel. Criaremos um modelo de machine learning capaz de classificar as tags do stack overflow.

O que foi aprendido?
- Importação dos dados
- Diferenças binaria, multiclass, multilabel
- Strings vs Classificação
- Treino e teste, zip das variáveis
- Vetorização dos textos
- Acurácia vs Hamming Loss
- BR, Chain, MLkNN
- Análise dos resultados

Palavras-chave: Modelo de Relevância Binária, Hamming Loss, ClassifierChain, Scikit-Multilearn, MLkNN, train_test_split, TF-IDF, OneVsRestClassifier, Exact Match.

# Explorando o problema

In [ ]:
uri = 'https://raw.githubusercontent.com/alura-cursos/alura_classificacao_multilabel/master/dataset/stackoverflow_perguntas.csv'

## 1. Importe os dados e exiba as 5 primeiras linhas

In [ ]:
import pandas as pd

perguntas = pd.read_csv(uri)
perguntas.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


# Classificadores

## 2. Quais os tipos de classificação?

- Classificação binária
- Classificação multiclasses
- Classificação multilabel

# Desafios de usar string como target

## 3. Quantas linhas tem nosso dataframe?

In [ ]:
perguntas.shape[0]

5408

## 4. Quantos tipos de tags tem na coluna tag?

In [ ]:
perguntas['Tags'].unique().shape[0]

37

In [ ]:
perguntas['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

# Explorando uma possível solução de utilizar strings como alvo

## 5. Crie uma lista com cada label única

In [ ]:
labels = list()

for tags in perguntas['Tags'].unique():
  for tag in tags.split():
    if tag not in labels:
      labels.append(tag)

labels

['node.js', 'jquery', 'html', 'angular']

# Transformando labels em colunas

## 6. Crie colunas para as labels

In [ ]:
node_js = list()

for i in perguntas['Tags']:
  if 'node.js' in i:
    node_js.append(1)
  else:
    node_js.append(0)

perguntas

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular
...,...,...
5403,Queria saber como pegar o total de cores de um...,jquery html
5404,"Boa noite, estou usando phonegap para fazer um...",html
5405,"Estou construindo um mini fórum, e nele, os us...",jquery html
5406,"Boa tarde, Estou para desenvolver um site na ...",html


## 7. Construa uma função de criação de colunas

In [ ]:
def nova_coluna(labels):
  for label in labels:
    lista = list()
    for linha in perguntas['Tags']:
      if label in linha:
        lista.append(1)
      else:
        lista.append(0)
    perguntas[label] = lista

In [ ]:
nova_coluna(labels)
perguntas.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular
1181,Estou tentando utilizar a mascara angular-inpu...,angular,0,0,0,1
2302,CODE CODE CODE Tenho esse código q...,html,0,0,1,0
791,formulário tem o button foram do então eu faç...,angular,0,0,0,1
2248,"Como pode ser visto no código, as variáveis de...",html,0,0,1,0
2393,Gostaria de saber como faço para substituir os...,html,0,0,1,0
277,"Dada uma data ex: 24/05/1982, como transformar...",html,0,0,1,0
4902,"Estou trabalhando em um site estático, onde te...",html node.js,1,0,1,0
4118,Estou com problema na migração do ionic beta.1...,angular,0,0,0,1
2092,Eu tenho a ideia de construir um rodapé fixo n...,html,0,0,1,0
3608,"Saudações dev's, eu declarei o schema de segui...",node.js,1,0,0,0


# Dados de treino e teste

## 8. Para que serve o train_test_split?

O train_test_split é o método que separa nossos dados em treino e teste.

## 9. Importe o train_test_split.

In [ ]:
from sklearn.model_selection import train_test_split

perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(
    perguntas['Perguntas'],
    perguntas['Tags']
)

# Zip para classificação multilabel

In [ ]:
lista1 = [1,2]
lista2 = [5,4]
lista_zip = zip(lista1, lista2)
print(list(lista_zip))

[(1, 5), (2, 4)]


## 10. Crie um zip com a lista de labels

In [ ]:
lista_zip_tags = list(zip(perguntas[labels[0]],
                          perguntas[labels[1]],
                          perguntas[labels[2]],
                          perguntas[labels[3]]))

perguntas['todas_tags'] = lista_zip_tags

perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


# TF-IDF

## 11. Passe a nova coluna todas tags como parâmetro

In [ ]:
from sklearn.model_selection import train_test_split

perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(
    perguntas['Perguntas'],
    perguntas['todas_tags'],
    test_size = 0.2,
    random_state = 123
)

## 12. O que é TF-IDF?

A principal característica do TF-IDF é ser uma pontuação proporcional à frequência da palavra no texto e equilibrada pela frequência no corpus, ou seja, palavras que se repetem muito tendem a ter pontuações menores e são menos relevantes no processo de classificação.

# Valorizando nossos textos

## 13. Importe o vetorizador TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetorizar = TfidfVectorizer(max_features = 5000, max_df = 0.85)
vetorizar

TfidfVectorizer(max_df=0.85, max_features=5000)

## 14. Vetorize de forma que o modelo de machine learning entenda.

In [ ]:
vetorizar.fit(perguntas['Perguntas'])
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_teste_tfidf = vetorizar.transform(perguntas_teste)
print(perguntas_treino_tfidf.shape)
print(perguntas_teste_tfidf.shape)

(4326, 5000)
(1082, 5000)


# Relevância Binária

Temos três perguntas que precisam ser classificadas. Algo parecido com o que fizemos no passado, classificando se era novela ou não. O nome daquela classificação era binária. Talvez eu possa usar isso aqui para cada coluna. O nome desse método é relevância binária. Ele vai criar um modelo para cada coluna. Temos a entrada das perguntas, que vai passar pelo modelo e classificar. No fim, tenho a classificação multilabel. Esse modelo resolve nosso problema. Vamos implementar.

# OneVsRest

## 15. Importe o OneVsRestClassifier

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

## 16. Construa o classificador

In [ ]:
regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)

classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino)

ValueError: ignored

## 17. Qual possível erro e como contorná-lo?

O modelo pede o uso de um array binário ou matriz esparsa. Para isso é necessário fazer uma modificação nos dados de treino e de teste.

# Realizando a primeira classificação

## 18. Transforme em array tags_treino

In [ ]:
import numpy as np

tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))
print(tags_treino_array)
print(type(tags_treino_array))

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]
<class 'numpy.ndarray'>


## 19. Treine o modelo novamente com a variável tags_treino_array

In [ ]:
regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_onevsrest = classificador_onevsrest.score(perguntas_teste_tfidf, tags_teste_array)
print('Resultado {0:.2f}%'.format(resultado_onevsrest * 100))

Resultado 41.68%


# Exact Match

## 20. Quantas combinações possíveis tem na coluna todas_tags?

In [ ]:
perguntas['todas_tags'].unique().shape[0]

13

# Hamming Loss

Diferente das métricas de acurácia, o hamming loss é melhor quanto mais próximo de zero. Se eu tenho, por exemplo, 0.9 de distância, isso é ruim. Entendendo isso, vamos colocar em prática na nossa previsão.

# Calculando o Hamming Loss

## 21. Importe o Hamming Loss.

In [ ]:
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador_onevsrest.predict(perguntas_teste_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_teste_array, previsao_onevsrest)
print('Hamming Loss {0:.2f}'.format(hamming_loss_onevsrest))

Hamming Loss 0.19


## 22. Calcule a correlação das tags.

In [ ]:
perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


# Classificação em cadeia

# Scikitmultilearn

## 23. Importe o Classifier Chain e treine o modelo

In [ ]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from skmultilearn.problem_transform import ClassifierChain

classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia = classificador_cadeia.score(perguntas_teste_tfidf, tags_teste_array)

previsao_cadeia = classificador_cadeia.predict(perguntas_teste_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, previsao_cadeia)

print('Hamming Loss {0:.2f}'.format(hamming_loss_cadeia))
print('Resultado {0:.2f}%'.format(resultado_cadeia * 100))

Hamming Loss 0.21
Resultado 49.82%


# Analisando os classificadores

## 24. Como escolher o melhor modelo?

Depende do contexto do problema. No nosso projeto o hamming loss foi uma métrica mais importante, portanto o modelo de Relevância binária faz mais sentido.

## 25. Quais as duas abordagens que podemos utilizar para classificação multilabel?

1. Transformação dos dados para classificação binária
2. Algoritmo adaptativo

# Para saber mais: relevância binária com skmultilearn

## 26. Como utilizar classificação de relevância binária com o skmultilearn?

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br = classificador_br.score(perguntas_teste_tfidf, tags_teste_array)

previsao_br = classificador_br.predict(perguntas_teste_tfidf)
hamming_loss_br = hamming_loss(tags_teste_array, previsao_br)

print('Hamming Loss {0:.2f}'.format(hamming_loss_br))
print('Resultado {0:.2f}%'.format(resultado_br * 100))

Hamming Loss 0.19
Resultado 41.68%


## Entendendo o ML-KNN

## 27. O que é KNN e como funciona?

KNN (K-Nearest Neighbor) é um algoritmo que classifica uma variável de acordo com a sua proximidade com outras variáveis treinadas previamente. 

## 28. O que é o ML-KNN e como funciona?

O ML-KNN é um complemento do algoritmo KNN que o complementa, através de calculos por "trás dos panos", confirmando a decisão tomada pelo KNN.

# Utilizando o ML-KNN

In [ ]:
# hamming_loss_mlknn = hamming_loss(tags_teste_array, previsao_mlknn)

## 29. Como importar o ML-KNN?

In [ ]:
!pip install scikit-learn==0.24.1

from skmultilearn.adapt import MLkNN

classificador_mlknn = MLkNN()
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_teste_tfidf, tags_teste_array)

previsao_mlknn = classificador_mlknn.predict(perguntas_teste_tfidf)
hamming_loss_mlknn = hamming_loss(tags_teste_array, previsao_mlknn)

print('Hamming Loss {0:.2f}'.format(hamming_loss_mlknn))
print('Resultado {0:.2f}%'.format(resultado_mlknn * 100))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Hamming Loss 0.25
Resultado 32.53%


## 30. Compare os algoritmos.

In [ ]:
print('Hamming Loss MLKNN {0:.2f}'.format(hamming_loss_mlknn))
print('Resultado MLKNN {0:.2f}%'.format(resultado_mlknn * 100))

Hamming Loss MLKNN 0.25
Resultado MLKNN 32.53%


In [ ]:
print('Hamming Loss cadeia {0:.2f}'.format(hamming_loss_cadeia))
print('Resultado cadeia {0:.2f}%'.format(resultado_cadeia * 100))

Hamming Loss cadeia 0.21
Resultado cadeia 49.82%


In [ ]:
print('Hamming Loss br {0:.2f}'.format(hamming_loss_br))
print('Resultado br {0:.2f}%'.format(resultado_br * 100))

Hamming Loss br 0.19
Resultado br 41.68%


# Verificando as classificações

## 31. Crie um dataframe para as classificações

In [ ]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao['perguntas'] = perguntas_teste.values
resultados_classificacao['tags_real'] = list(tags_teste)
resultados_classificacao['BR'] = list(previsao_br.toarray())
resultados_classificacao['cadeia'] = list(previsao_cadeia.toarray())
resultados_classificacao['mlknn'] = list(previsao_mlknn.toarray())
resultados_classificacao

,perguntas,tags_real,BR,cadeia,mlknn
0,estou com conflito entre o CODE e os CODE ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 1, 0)","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 1, 1, 0]"
2,Recentemente fiz um refactor do meu código par...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,"Olá, em minha função tem o evento CODE que de...","(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
...,...,...,...,...,...
1077,Estou a desenvolver um website em jQuery. E at...,"(0, 1, 0, 0)","[0, 1, 1, 0]","[0.0, 1.0, 1.0, 0.0]","[0, 0, 1, 0]"
1078,Estou usando este plugin - jquery autocomplete...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1079,"Tenho o seguinte jQuery: CODE Nisto, quanti...","(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1080,Estou usando o SimpleModal Contact Form de Eri...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"


## 32. Visualize as linhas através do iloc.

In [ ]:
resultados_classificacao.iloc[1]

perguntas    Estou fazendo um site que eu sou obrigado a us...
tags_real                                         (0, 0, 1, 0)
BR                                                [0, 0, 1, 0]
cadeia                                    [0.0, 0.0, 1.0, 0.0]
mlknn                                             [0, 1, 1, 0]
Name: 1, dtype: object

In [ ]:
resultados_classificacao['perguntas'][1]

'Estou fazendo um site que eu sou obrigado a usar HTML, CSS e JavaScript no máximo. Só que muitas coisas que eu preciso do PhP eu não posso usar. Sendo assim eu estive pesquisando por sinonimo de CODE  e encontrei o seguinte, que ajudou em partes:  CODE   Tá, mas qual é o problema? Ele cria uma barra de rolagem do lado. Eu sei como remover com CSS, mas o texto que foi inserido rola só dentro da área da CODE  e eu queria que fizesse como se o texto inserido fizesse parte da página, como funciona no CODE  do PhP. A questão é, como fazer a mecânica do JavaScript nesse caso ficar igual ao do PhP?  Print para exemplificar: https://imgur.com/a/V9AGLgB '

# Conclusão

Construímos um modelo de classifciação multilabel para as tags do Stack Overflow. Fizemos a importação dos dados, entendemos as diferenças entre classificação binária, multiclasse e multilabel. Depois separamos nossos dados em treino e teste. Fizemos a vetorização dos textos. Entendemos qual melhor modelo a ser escolhido analisando as métricas de acurácia e hamming loss e optamos pelo modelo de Relevância Binária.
